In [1]:
import pandas as pd
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors


# Download necessary NLTK packages
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('words')

KeyboardInterrupt: 

ImportError: Runtime and compile-time PyObject size do not match.

In [ ]:
stop_words = set(nltk.corpus.stopwords.words('english'))

### Resume

In [ ]:
df_resumes = pd.read_csv('PreProcessing/processed_data/Resume_proc_lemm.csv')
df_resumes.head()

In [ ]:
df_resumes.info()

### Job Description

In [ ]:
df_jobs = pd.read_csv('PreProcessing Jobs\processed_data\Job_proc_lemm.csv')
df_jobs.head()

In [ ]:
df_jobs.info()

### Similarity

In [ ]:
tfidf_vect = TfidfVectorizer(
    lowercase=True,
    binary=False,
    max_features=10000,
    stop_words=list(stop_words),   # come prima, valuta se serve
    ngram_range=(1, 6),            # unigrams fino a 6-grams
    max_df=0.8,
    min_df=2
)

In [ ]:
cv_texts = df_resumes['Resume_str'].tolist()
job_texts = df_jobs['description'].tolist()

In [ ]:
corpus = cv_texts + job_texts

In [ ]:
tfidf_matrix = tfidf_vect.fit_transform(corpus)

In [ ]:
# Dividi la matrice
cv_vectors = tfidf_matrix[:len(cv_texts)]
job_vectors = tfidf_matrix[len(cv_texts):]

In [ ]:
top_n = 10
knn = NearestNeighbors(n_neighbors=top_n, metric='cosine')
knn.fit(cv_vectors)

In [ ]:
distances, indices = knn.kneighbors(job_vectors)

In [ ]:
results = []
for job_idx, (dist_list, idx_list) in enumerate(zip(distances, indices)):
    job_real_id = df_jobs.iloc[job_idx]['job_id']

    for rank, (cv_idx, dist) in enumerate(zip(idx_list, dist_list)):
        similarity = 1 - dist  # sim = 1 - distanza
        cv_real_id = df_resumes.iloc[cv_idx]['ID']
        
        results.append({
            'job_id': job_real_id,
            'cv_id': cv_real_id,
            'similarity_score': similarity,
            'rank': rank + 1
        })

df_matches = pd.DataFrame(results)
print(df_matches.head(10))